In [ ]:
%pip install zmq web3 cryptography

In [2]:
import zmq
from web3 import Web3

from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.hashes import SHA256
from cryptography.hazmat.backends import default_backend

In [3]:
def encrypt(message):
    with open("keys/public.pem", "rb") as f:
        public_key = serialization.load_pem_public_key(
            f.read(),
            backend=default_backend()
        )

    encrypted_message = public_key.encrypt(
        message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=SHA256()),
            algorithm=SHA256(),
            label=None
        )
    )

    return encrypted_message

In [4]:
def decrypt(encrypted_message): 
    with open("keys/private.pem", "rb") as f:
        private_key = serialization.load_pem_private_key(
            f.read(),
            password=None,
            backend=default_backend()
        )

    decrypted_message = private_key.decrypt(
        encrypted_message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=SHA256()),
            algorithm=SHA256(),
            label=None
        )
    )

    return decrypted_message

In [5]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:9944"))

with open("contract/address", "r") as file:
    address = file.read()

with open("contract/abi", "r") as file:
    abi = file.read()

contract_instance = w3.eth.contract(address=address, abi=abi)

In [ ]:
class TEE:
    def __init__(self, client_address="tcp://*:9002", server_address="tcp://127.0.0.1:9003"):
        self.context = zmq.Context()

        # Client Communication
        self.client_socket = self.context.socket(zmq.REP)  # REPLY to client
        self.client_socket.bind(client_address)

        # Server Communication
        self.server_socket = self.context.socket(zmq.REQ)  # REQUEST to server
        self.server_socket.connect(server_address)

    def forward_to_server(self, data):
        """Forward the model received from the client to the server."""
        print("Forwarding model to server...")
        self.server_socket.send(data)
        return self.server_socket.recv()  # Receive response from server

    def forward_to_client(self, data):
        """Forward the aggregated model from the server to the client."""
        print("Forwarding aggregated model to client...")
        self.client_socket.send(data)

    def run(self):
        print("TEE is running...")
        while True:
            # Receive model from client
            client_message = self.client_socket.recv()
            print("Received model from client.")

            # Forward to server and get the response (aggregated model or ACK)
            server_response = self.forward_to_server(client_message)

            # Send the server response back to the client
            self.forward_to_client(server_response)

if __name__ == "__main__":
    tee_processor = TEE()
    tee_processor.run()
